In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
!pip install accelerate openai tiktoken transformers torch  retry bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.wh

## Hàm tạo Prompt

In [12]:

def create_prompt(text):
  prompt = f"""
Analyze the following scientific text as an expert in natural language processing and information extraction. Without relying on predefined categories, identify the key concepts, entities, and relationships between them that are crucial for understanding the scientific content. Your analysis should capture the essence of the research described, including any methods, tasks, materials, or metrics mentioned.

Present your findings in a structured format that clearly distinguishes between:
1. ner: Important entities or concepts as objects. Each object has a text and a type.
2. relations: Significant relationships or interactions between these entities as objects. Each object has a head, a tail and a type
Directionality is taken into account except for symmetric relation types.
3. types: provide a dictionary of types and their meanings.

Ensure your response is based solely on the information explicitly stated in the text, avoiding any inferences or predictions beyond what is directly presented.
Use your expertise to determine the most appropriate labels and categories based on the content of the text.


Text to analyze:
English is shown to be trans-context-free on the basis of coordinations of the respectively type that involve strictly syntactic cross-serial agreement. The agreement in question involves number in nouns and reflexive pronouns and is syntactic rather than semantic in nature because grammatical number in English, like grammatical gender in languages such as French, is partly arbitrary. The formal proof, which makes crucial use of the Interchange Lemma of Ogden et al., is so constructed as to be valid even if English is presumed to contain grammatical sentences in which respectively operates across a pair of coordinate phrases one of whose members has fewer conjuncts than the other; it thus goes through whatever the facts may be regarding constructions with unequal numbers of conjuncts in the scope of respectively, whereas other arguments have foundered on this problem.
Extracted results:
{{
  "ner": [
    {{"text": "English", "type": "Material"}},
    {{"text": "coordinations", "type": "OtherScientificTerm"}},
    {{"text": "cross-serial agreement", "type": "OtherScientificTerm"}},
    {{"text": "agreement", "type": "Generic"}},
    {{"text": "number", "type": "OtherScientificTerm"}},
    {{"text": "nouns and reflexive pronouns", "type": "OtherScientificTerm"}},
    {{"text": "grammatical number", "type": "OtherScientificTerm"}},
    {{"text": "English", "type": "Material"}},
    {{"text": "grammatical gender", "type": "OtherScientificTerm"}},
    {{"text": "French", "type": "Material"}},
    {{"text": "Interchange Lemma", "type": "Method"}},
    {{"text": "English", "type": "Material"}}
  ],
  "relations": [
    {{
      "text": "number",
      "tail": "nouns and reflexive pronouns",
      "type": "CONJUNCTION"
    }},
    {{
      "head": "grammatical gender",
      "tail": "French",
      "type": "FEATURE-OF"
    }},
    {{
      "head": "English",
      "tail": "French",
      "type": "HYPONYM-OF"
    }}
  ],
  "types": {{
    "Method": "Methods , models, systems to use, or tools, components of a system, frameworks.",
    "Material": "Data, datasets, resources, Corpus, Knowledge base.",
    "Other Scientific Terms": "Phrases that are a scientific terms but do not fall into any of the above classes.",
    "Generic": "General terms or pronouns that may refer to a entity but are not themselves informative, often used as connection words.",
    "FEATURE-OF": "attribute or characteristic",
    "HYPONYM-OF": "specific type or instance",
    "CONJUNCTION":"related or used together."
  }}
}}
IMPORTANT:
- Format your response as a JSON-like structure.
- The "text", "head", "tail" and "type" field for each entity or relation must always be a single string, never an array.
- If an entity consists of multiple words or phrases, combine them into a single string.
- Do not include any information other than the required JSON structure.
- No clarification is needed except for the types dictionary


Now, please extract the relationships from the following text:
  Text: {text}

  Extracted Results:

  """
  return prompt


# Chọn ngẫu nhiên mẫu

In [6]:
import json
def read_json_file(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line.strip()))
    return data
import random


def get_random_samples(input_list, num_samples=30):
    """
    Get random samples from the input list.

    Args:
    input_list (list): The list to sample from.
    num_samples (int): The number of samples to return. Default is 20.

    Returns:
    list: A list of random samples.
    """
    if len(input_list) < num_samples:
        raise ValueError(f"Input list must have at least {num_samples} elements.")

    return random.sample(input_list, num_samples)
file_path = '/content/gdrive/My Drive/llama3_data/SciErc/scierc.json'
json_data = read_json_file(file_path)

last_samples = get_random_samples(json_data)
def create_text_from_sample(data):
    # Ghép các từ trong mỗi câu
    sentences = [' '.join(sentence) for sentence in data['sentences']]

    # Ghép các câu thành một đoạn văn bản
    full_text = ' '.join(sentences)

    return full_text
def extract_entities_and_relations(data):
    all_tokens = [token for sentence in data['sentences'] for token in sentence]

    entities = []
    for ner_sentence in data['ner']:
        for start, end, entity_type in ner_sentence:
            entity_text = ' '.join(all_tokens[start:end+1])
            entities.append({"text": entity_text, "type": entity_type})

    relations = []
    for relation_sentence in data['relations']:
        for head_start, head_end, tail_start, tail_end, relation_type in relation_sentence:
            head_text = ' '.join(all_tokens[head_start:head_end+1])
            tail_text = ' '.join(all_tokens[tail_start:tail_end+1])
            relations.append({
                "head": head_text,
                "tail": tail_text,
                "type": relation_type
            })

    return {
        "ner": entities,
        "relations": relations
    }



## Hàm đánh giá kết quả

In [7]:
from difflib import SequenceMatcher
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import numpy as np
from scipy.spatial.distance import cosine



semantic_check_model_name = "bert-base-uncased"
semantic_check_tokenizer = AutoTokenizer.from_pretrained(semantic_check_model_name)
semantic_check_model = AutoModel.from_pretrained(semantic_check_model_name)

relation_meanings = {
    "USED-FOR": "utilization or application",
    "FEATURE-OF": "attribute or characteristic",
    "HYPONYM-OF": "specific type or instance",
    "PART-OF": "component or element",
    "COMPARE": "contrast or evaluation",
    "CONJUNCTION": "related or used together"
}

ner_meanings = {
    "Method": "Methods , models, systems to use, or tools, components of a system, frameworks.",
    "Material": "Data, datasets, resources, Corpus, Knowledge base.",
    "Other Scientific Terms": "Phrases that are a scientific terms but do not fall into any of the above classes.",
    "Generic": "General terms or pronouns that may refer to a entity but are not themselves informative, often used as connection words.",
    "Evaluation Metric": "Metrics, measures, or entities that can express quality of a system/method.",
    "Task": "Applications, problems to solve, systems to construct."
  }


def get_word_embedding(word):

    inputs = semantic_check_tokenizer(word, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = semantic_check_model(**inputs)

    return outputs.last_hidden_state[0][0].numpy()

def semantic_similarity(word1, word2):
    emb1 = get_word_embedding(word1)
    emb2 = get_word_embedding(word2)
    return 1 - cosine(emb1, emb2)

def text_similarity(text1, text2):
    #print(text1)
    #print(text2)
    return SequenceMatcher(None, text1.lower(), text2.lower()).ratio()


def calculate_ner_similarity(pred, gt, sem_dict):
    text_sim = 1 if text_similarity(pred['text'], gt['text']) > 0.4 else 0
    type_sim = semantic_similarity(sem_dict.get(pred['type'], pred['type']), ner_meanings.get(gt['type'], gt['type']))
    return (text_sim + type_sim) / 2

def calculate_relation_similarity(pred, gt, sem_dict):
    head_sim = 1 if text_similarity(pred['head'], gt['head']) > 0.4 else 0
    tail_sim = 1 if text_similarity(pred['tail'], gt['tail']) > 0.4 else 0
    type_sim = semantic_similarity(sem_dict.get(pred['type'], pred['type']), relation_meanings.get(gt['type'].lower(), gt['type']))
    return (head_sim + tail_sim + type_sim) / 3

def calculate_f1(predicted, ground_truth, sem_dict, task='ner', threshold=0.7):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    pred_items = predicted[task]
    gt_items = ground_truth[task]

    similarity_func = calculate_ner_similarity if task == 'ner' else calculate_relation_similarity

    for pred in pred_items:
        best_score = 0
        for gt in gt_items:
            similarity = similarity_func(pred, gt, sem_dict)
            best_score = max(best_score, similarity)

        if best_score >= threshold:
            true_positives += 1
        else:
            false_positives += 1

    for gt in gt_items:
        best_score = 0
        for pred in pred_items:
            similarity = similarity_func(pred, gt, sem_dict)
            best_score = max(best_score, similarity)

        if best_score < threshold:
            false_negatives += 1

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

## Xử lý output

In [8]:
def parse_llm_output(output):
    # Tìm phần JSON trong đầu ra của LLM
    json_start = output.find('{')
    json_end = output.rfind('}') + 1
    json_str = output[json_start:json_end]
    parsed_json = json.loads(json_str)
    return parsed_json



In [9]:
model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

# Chạy từng sample

In [10]:
def run_each_samples(sample_data):
  text = create_text_from_sample(sample_data)
  ground_truth = extract_entities_and_relations(sample_data)

  prompt = create_prompt(text)
  #print(prompt)
  messages = [
    {"role": "system", "content": prompt },
  ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]
  outputs = model.generate(
    input_ids,
    max_new_tokens=3000,
    eos_token_id=terminators,
    do_sample=False,
    temperature=0.0,
  )
  response = outputs[0][input_ids.shape[-1]:]
  llm_output = tokenizer.decode(response, skip_special_tokens=True)
  print(llm_output)
  llm_results = parse_llm_output(llm_output)
  sem_dict = llm_results["types"]
  print(llm_results)
  return {
      "ner": calculate_f1(llm_results, ground_truth,sem_dict, task='ner'),
      "relations": calculate_f1(llm_results, ground_truth,sem_dict, task='relations')
  }



In [13]:

total_precision = 0
total_recall = 0
total_f1 = 0
total_count
for i, sample in enumerate(last_samples, 1):
    try:
      res = run_each_samples(sample)
    except Exception as e:
      print(f"Error in sample {i}")
      print(e)
      continue
    print(res)
    ner_precision, ner_recall, ner_f1 = res['ner']
    relations_precision, relations_recall, relations_f1 = res['relations']
    total_precision += ner_precision + relations_precision
    total_recall += ner_recall + relations_recall
    total_f1 += ner_f1 + relations_f1
    total_count += 1
    print(total_precision)
    print(total_recall)
    print(total_f1)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Here are the extracted results in the required JSON-like structure:

{
  "ner": [
    {"text": "Chinese-to-English SMT model", "type": "Method"},
    {"text": "word sense disambiguation performance", "type": "OtherScientificTerm"},
    {"text": "standard WSD evaluation methodology", "type": "Method"},
    {"text": "datasets", "type": "Material"},
    {"text": "Senseval-3 Chinese lexical sample task", "type": "Material"},
    {"text": "WSD", "type": "OtherScientificTerm"},
    {"text": "BLEU scores", "type": "OtherScientificTerm"},
    {"text": "statistical machine translation", "type": "OtherScientificTerm"},
    {"text": "SMT models", "type": "Method"},
    {"text": "dedicated WSD models", "type": "Method"},
    {"text": "WSD accuracy", "type": "OtherScientificTerm"},
    {"text": "current typical SMT models", "type": "Method"},
    {"text": "dedicated WSD models", "type": "Method"},
    {"text": "speculative claims", "type": "OtherScientificTerm"}
  ],
  "relations": [
    {
      "h

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'ner': (1.0, 0.875, 0.9333333333333333), 'relations': (0.3333333333333333, 0.2727272727272727, 0.3)}
1.3333333333333333
1.1477272727272727
1.2333333333333334
Here are the extracted results:

{
  "ner": [
    {"text": "bit-vector-based CKY-style parser", "type": "Method"},
    {"text": "CKY-style parser", "type": "Method"},
    {"text": "context-free parsing", "type": "OtherScientificTerm"},
    {"text": "parse forest representation", "type": "OtherScientificTerm"},
    {"text": "complete set of possible analyses", "type": "OtherScientificTerm"},
    {"text": "large treebank grammars", "type": "Material"},
    {"text": "long input sentences", "type": "Material"},
    {"text": "bit-vector operations", "type": "Method"},
    {"text": "basic parsing operations", "type": "OtherScientificTerm"},
    {"text": "parser", "type": "Method"},
    {"text": "analyses", "type": "Generic"}
  ],
  "relations": [
    {
      "head": "parser",
      "tail": "bit-vector-based CKY-style parser",
      "ty

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

{'ner': (0.5555555555555556, 1.0, 0.7142857142857143), 'relations': (0.0, 0.0, 0)}
26.40540322579303
25.8901760228076
25.570602163909193
Here are the extracted results:

{
  "ner": [
    {"text": "bispectral photo-metric stereo", "type": "Method"},
    {"text": "fluorescence", "type": "OtherScientificTerm"},
    {"text": "shape reconstruction", "type": "Task"},
    {"text": "fluorescence", "type": "OtherScientificTerm"},
    {"text": "natural gems and corals", "type": "Material"},
    {"text": "fluorescent dyes used in clothing", "type": "Material"},
    {"text": "wavelength-shifting behavior", "type": "OtherScientificTerm"},
    {"text": "fluorescent materials", "type": "OtherScientificTerm"},
    {"text": "light", "type": "OtherScientificTerm"},
    {"text": "wavelengths", "type": "OtherScientificTerm"},
    {"text": "fluorescence", "type": "OtherScientificTerm"},
    {"text": "algorithms", "type": "OtherScientificTerm"},
    {"text": "computer vision", "type": "Field"},
    {"text":

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'ner': (0.9130434782608695, 0.875, 0.8936170212765957), 'relations': (0.18181818181818182, 0.18181818181818182, 0.18181818181818182)}
27.50026488587208
26.946994204625785
26.64603736700397
Here are the extracted results in the required JSON-like structure:

{
  "ner": [
    {"text": "speech-based depression detection", "type": "Material"},
    {"text": "depression detection", "type": "Material"},
    {"text": "noise", "type": "OtherScientificTerm"},
    {"text": "reverberation", "type": "OtherScientificTerm"},
    {"text": "MFCCs", "type": "OtherScientificTerm"},
    {"text": "DOCCs", "type": "OtherScientificTerm"},
    {"text": "2014 AudioVisual Emotion Recognition Challenge", "type": "Material"},
    {"text": "AVEC", "type": "Material"},
    {"text": "additive noise", "type": "OtherScientificTerm"},
    {"text": "evaluation metrics", "type": "OtherScientificTerm"},
    {"text": "artificial neural networks", "type": "Method"},
    {"text": "support vector regression", "type": "Method

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'ner': (0.875, 0.7777777777777778, 0.823529411764706), 'relations': (0.5, 0.2222222222222222, 0.30769230769230765)}
28.87526488587208
27.946994204625785
27.777259086460983
Here are the extracted results:

{
  "ner": [
    {"text": "approach", "type": "Method"},
    {"text": "intrinsic texture properties", "type": "OtherScientificTerm"},
    {"text": "albedo", "type": "OtherScientificTerm"},
    {"text": "shading", "type": "OtherScientificTerm"},
    {"text": "normal", "type": "OtherScientificTerm"},
    {"text": "scenes", "type": "Material"},
    {"text": "multiple view acquisition", "type": "Method"},
    {"text": "unknown illumination conditions", "type": "OtherScientificTerm"},
    {"text": "intrinsic textures", "type": "OtherScientificTerm"},
    {"text": "pixel-resolution surface textures", "type": "OtherScientificTerm"},
    {"text": "intrinsic appearance parameters", "type": "OtherScientificTerm"},
    {"text": "previous video relighting methods", "type": "Method"},
    {"text"

## Kết quả

In [15]:
total_precision /= 2*total_count
total_recall /= 2*total_count
total_f1 /= 2*total_count

print(f"Average Precision: {total_precision:.4f}")
print(f"Average Recall: {total_recall:.4f}")
print(f"Average F1 Score: {total_f1:.4f}")


Average Precision: 0.5607
Average Recall: 0.5389
Average F1 Score: 0.5370
